In [39]:
import pandas as pd
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine, func, inspect, text

In [40]:
df = pd.read_csv("NY-House-Dataset_CLEANNNN.csv")
df.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,Distance,PRICE_BY_SQFT,Borough,House_Type
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,...,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483,2.950061,225.000000,NaN,Condo
1,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,...,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109,18.901470,129.032258,Staten Island,House
2,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,...,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613,2.961863,155.056180,NaN,Condo
3,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,...,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856,3.111680,3880.070547,NaN,Townhouse
4,Brokered by Sowae Corp,House for sale,690000,5,2.000000,4004.0,584 Park Pl,"Brooklyn, NY 11238","584 Park PlBrooklyn, NY 11238",United States,...,Kings County,Brooklyn,Park Place,"584 Park Pl, Brooklyn, NY 11238, USA",40.674363,-73.958725,4.076335,172.327672,Brooklyn,House


In [41]:
engine = create_engine("sqlite:///NYC_House_db.sqlite")

df.to_sql("NYC_Houses", con=engine, if_exists="replace", index=False)

4141

In [43]:
# explore and understand the data

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

NYC_Houses
BROKERTITLE TEXT
TYPE TEXT
PRICE BIGINT
BEDS BIGINT
BATH FLOAT
PROPERTYSQFT FLOAT
ADDRESS TEXT
STATE TEXT
MAIN_ADDRESS TEXT
ADMINISTRATIVE_AREA_LEVEL_2 TEXT
LOCALITY TEXT
SUBLOCALITY TEXT
STREET_NAME TEXT
LONG_NAME TEXT
FORMATTED_ADDRESS TEXT
LATITUDE FLOAT
LONGITUDE FLOAT
Distance FLOAT
PRICE_BY_SQFT FLOAT
Borough TEXT
House_Type TEXT



In [19]:
price = 500000

# allow the user to choose price
if price == "All":
    where_clause = "1=1"
else:
    where_clause = f"PRICE <= '{price}'"

query = f"""
        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            PRICE <= '500000';



In [20]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,Distance,PRICE_BY_SQFT,Borough,House_Type
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,...,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483,2.950061,225.000000,None,Condo
1,Brokered by Sowae Corp,House for sale,260000,4,2.0,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,...,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109,18.901470,129.032258,Staten Island,House
2,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,...,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613,2.961863,155.056180,None,Condo
3,Brokered by Pantiga Group Inc.,Co-op for sale,265000,1,1.0,750.0,875 Morrison Ave Apt 3M,"Bronx, NY 10473","875 Morrison Ave Apt 3MBronx, NY 10473",Bronx County,...,East Bronx,Morrison Avenue,Parking lot,"Parking lot, 875 Morrison Ave #3m, Bronx, NY 1...",40.821586,-73.874089,7.053388,353.333333,None,Co-op
4,Brokered by CENTURY 21 MK Realty,Co-op for sale,440000,2,1.0,978.0,1350 Ocean Pkwy Apt 5G,"Brooklyn, NY 11230","1350 Ocean Pkwy Apt 5GBrooklyn, NY 11230",New York,...,Brooklyn,Midwood,1350,"1350 Ocean Pkwy #5g, Brooklyn, NY 11230, USA",40.615738,-73.969694,8.139742,449.897751,None,Co-op


In [21]:
num_beds = 3

# allow the user to select how many bedrooms
if num_beds == "All":
    where_clause = "1=1"
else:
    where_clause = f"BEDS <= '{num_beds}'"

query = f"""
        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            BEDS <= '3';



In [22]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,Distance,PRICE_BY_SQFT,Borough,House_Type
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.000000,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,...,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483,2.950061,225.000000,None,Condo
1,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.000000,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,...,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613,2.961863,155.056180,None,Condo
2,Brokered by Douglas Elliman - 575 Madison Ave,Condo for sale,899500,2,2.0,2184.207862,157 W 126th St Unit 1B,"New York, NY 10027","157 W 126th St Unit 1BNew York, NY 10027",New York,...,New York,Manhattan,157,"157 W 126th St #1b, New York, NY 10027, USA",40.809448,-73.946777,5.480580,411.819779,None,Condo
3,Brokered by Pantiga Group Inc.,Co-op for sale,265000,1,1.0,750.000000,875 Morrison Ave Apt 3M,"Bronx, NY 10473","875 Morrison Ave Apt 3MBronx, NY 10473",Bronx County,...,East Bronx,Morrison Avenue,Parking lot,"Parking lot, 875 Morrison Ave #3m, Bronx, NY 1...",40.821586,-73.874089,7.053388,353.333333,None,Co-op
4,Brokered by CENTURY 21 MK Realty,Co-op for sale,440000,2,1.0,978.000000,1350 Ocean Pkwy Apt 5G,"Brooklyn, NY 11230","1350 Ocean Pkwy Apt 5GBrooklyn, NY 11230",New York,...,Brooklyn,Midwood,1350,"1350 Ocean Pkwy #5g, Brooklyn, NY 11230, USA",40.615738,-73.969694,8.139742,449.897751,None,Co-op


In [23]:
num_baths = 2

# allow the user to choose how many baths
if num_baths == "All":
    where_clause = "1=1"
else:
    where_clause = f"BATH <= '{num_baths}'"

query = f"""
        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            BATH <= '2';



In [24]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,Distance,PRICE_BY_SQFT,Borough,House_Type
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.0,1400.000000,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,...,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483,2.950061,225.000000,None,Condo
1,Brokered by Sowae Corp,House for sale,260000,4,2.0,2015.000000,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,...,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109,18.901470,129.032258,Staten Island,House
2,Brokered by COMPASS,Condo for sale,69000,3,1.0,445.000000,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,...,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613,2.961863,155.056180,None,Condo
3,Brokered by Sowae Corp,House for sale,690000,5,2.0,4004.000000,584 Park Pl,"Brooklyn, NY 11238","584 Park PlBrooklyn, NY 11238",United States,...,Kings County,Brooklyn,Park Place,"584 Park Pl, Brooklyn, NY 11238, USA",40.674363,-73.958725,4.076335,172.327672,Brooklyn,House
4,Brokered by Douglas Elliman - 575 Madison Ave,Condo for sale,899500,2,2.0,2184.207862,157 W 126th St Unit 1B,"New York, NY 10027","157 W 126th St Unit 1BNew York, NY 10027",New York,...,New York,Manhattan,157,"157 W 126th St #1b, New York, NY 10027, USA",40.809448,-73.946777,5.480580,411.819779,None,Condo


In [27]:
house_type = 'House'

# allow the user to select a house type
if house_type == "All":
    where_clause = "1=1"
else:
    where_clause = f"House_Type == '{house_type}'"

query = f"""
        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            House_Type == 'House';



In [28]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,Distance,PRICE_BY_SQFT,Borough,House_Type
0,Brokered by Sowae Corp,House for sale,260000,4,2.0,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,...,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109,18.901470,129.032258,Staten Island,House
1,Brokered by Sowae Corp,House for sale,690000,5,2.0,4004.0,584 Park Pl,"Brooklyn, NY 11238","584 Park PlBrooklyn, NY 11238",United States,...,Kings County,Brooklyn,Park Place,"584 Park Pl, Brooklyn, NY 11238, USA",40.674363,-73.958725,4.076335,172.327672,Brooklyn,House
2,Brokered by Connie Profaci Realty,House for sale,16800000,8,16.0,33000.0,177 Benedict Rd,"Staten Island, NY 10304","177 Benedict RdStaten Island, NY 10304",United States,...,Richmond County,Staten Island,Benedict Road,"177 Benedict Rd, Staten Island, NY 10304, USA",40.595002,-74.106424,12.972564,509.090909,Staten Island,House
3,Brokered by Radiant Estates LLC,House for sale,639999,3,2.0,1260.0,27 Clove Way,"Staten Island, NY 10301","27 Clove WayStaten Island, NY 10301",United States,...,Richmond County,Staten Island,Clove Way,"27 Clove Way, Staten Island, NY 10301, USA",40.620740,-74.111436,11.953013,507.935714,None,House
4,Brokered by Elizabeth Marra,House for sale,750000,2,2.0,1330.0,280 Loretto St,"Staten Island, NY 10307","280 Loretto StStaten Island, NY 10307",United States,...,Richmond County,Staten Island,Loretto Street,"280 Loretto St, Staten Island, NY 10307, USA",40.503979,-74.239655,22.363029,563.909774,None,House


In [47]:
house_type = 'House'
price = 500000
num_beds = 3
num_baths = 2
price_by_sqft = 250
distance = 12



if house_type == "All":
    where_clause1 = "1=1"
else:
    where_clause1 = f"House_Type == '{house_type}'"
    
if price == "All":
    where_clause2 = "1=1"
else:
    where_clause2 = f"PRICE <= {price}"
    
if num_beds == "All":
    where_clause3 = "1=1"
else:
    where_clause3 = f"BEDS <= {num_beds}"
    
if num_baths == "All":
    where_clause4 = "1=1"
else:
    where_clause4 = f"BATH <= {num_baths}"

if price_by_sqft == "All":
    where_clause5 = "1=1"
else:
    where_clause5 = f"PRICE_BY_SQFT <= {price_by_sqft}"
    
if distance == "All":
    where_clause6 = "1=1"
else:
    where_clause6 = f"Distance <= {distance}"
    
query = f"""
        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            {where_clause1} AND {where_clause2} AND {where_clause3} AND {where_clause4} AND {where_clause5} AND {where_clause6};
"""

print(query)


        SELECT
            *
        FROM
            NYC_Houses
        WHERE
            House_Type == 'House' AND PRICE <= 500000 AND BEDS <= 3 AND BATH <= 2 AND PRICE_BY_SQFT <= 250 AND Distance <= 12;



In [48]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,...,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE,Distance,PRICE_BY_SQFT,Borough,House_Type
0,Brokered by Sowae Corp,House for sale,314000,3,1.0,1572.000000,850 E 39th St,"Brooklyn, NY 11210","850 E 39th StBrooklyn, NY 11210",United States,...,Kings County,Brooklyn,East 39th Street,"850 E 39th St, Brooklyn, NY 11210, USA",40.636237,-73.939679,6.524774,199.745547,Brooklyn,House
1,Brokered by Sowae Corp,House for sale,245000,3,1.0,2184.207862,2942 Lurting Ave,"Bronx, NY 10469","2942 Lurting AveBronx, NY 10469",United States,...,Bronx County,The Bronx,Lurting Avenue,"2942 Lurting Ave, Bronx, NY 10469, USA",40.869011,-73.856444,10.413024,112.168812,None,House
2,Brokered by Sowae Corp,House for sale,265000,3,1.0,2184.207862,135-38 224th St,"Springfield Gardens, NY 11413","135-38 224th StSpringfield Gardens, NY 11413",United States,...,Queens County,Queens,224th Street,"135-38 224th St, Jamaica, NY 11413, USA",40.675380,-73.747257,10.560290,121.325449,None,House
3,Brokered by Sowae Corp,House for sale,285000,2,1.0,1380.000000,102-15 215 St,"Queens Village, NY 11429","102-15 215 StQueens Village, NY 11429",United States,...,Queens County,Queens,215th Street,"102-15 215th St, Jamaica, NY 11429, USA",40.712833,-73.743578,10.111472,206.521739,None,House
4,Brokered by Sowae Corp,House for sale,420000,3,1.0,2184.207862,90-07 107th Ave,"Ozone Park, NY 11417","90-07 107th AveOzone Park, NY 11417",United States,...,Queens County,Queens,107th Avenue,"90-07 107th Ave, Queens, NY 11417, USA",40.678476,-73.847686,5.831607,192.289391,None,House
